# HC Column Height Study

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline
plt.style.use('seaborn')
sns.set_style('darkgrid')

In [3]:
df = pd.read_csv(r'alldata_Edmundson.csv') 

df.drop(['Depth_1', 'Trap_Fill%', 'Class1', 'Class2', 'Class3'], axis=1, inplace=True)
df.head()

,Depth,HC_Column,Trap_height,Trap_Fill_2%
0,1392,44,90,49
1,1961,20,49,41
2,2450,46,100,46
3,3209,38,77,49
4,199,76,154,49


In [ ]:
## bins used with pd.cut(): [1, 2, 3, 4] indicates (1,2], (2,3], (3,4].
## Trap height bins : (0,150],(150,300],(300,1000]
th_bins = [0,150,300,1000]
th_labels = ['A','B','C']
## Overburden bins
ob_bins = [0,1500,3000,10000]
ob_labels = ['1','2','3']
## Trap fill percent bins
tf_bins = [0,50,75,99,100]
tf_labels = ['A','B','C','D']

## Create columns binning the data into categories
df['TH'] = pd.cut(df['Trap_height'], bins=th_bins, labels=th_labels)
df['OB'] = pd.cut(df['Depth'], bins=ob_bins, labels=ob_labels)
df['TF'] = pd.cut(df['Trap_Fill_2%'], bins=tf_bins, labels=tf_labels)

## Combine the individual categories
## Combine cols 'TH' and 'OB' into TH_OB
df['TH_OB'] = df['TH'].str.cat(df['OB'],sep="")
## Combine cols 'TH_OB' and 'TF' into 'TH_OB_TF'
df['TH_OB_TF'] = df['TH_OB'].str.cat(df['TF'],sep="")

## Copy the category columns - TH_OB, etc will be modified later
df['th_ob'] = df['TH_OB']
df['th_ob_tf'] = df['TH_OB_TF']
df['th'] = df['TH']

In [ ]:
## Change oject types to Category so the dictionary swap can work
for col in ['TH_OB', 'TH_OB_TF', 'th_ob', 'th_ob_tf']:
    df[col] = df[col].astype('category')

## Create lists containing unique category labels
th_ob_labels = list(df['TH_OB'].unique())
th_ob_tf_labels = list(df.TH_OB_TF.unique())

## Number of total outcomes
outcomes = len(df.index)

In [ ]:
## Create dictionarys relating the labels with the number of elements in each label

d_th = {}
for i in th_labels:
    d_th[i] = len(df[df.TH == [i]])

d_th_ob = {}
for i in th_ob_labels:
    d_th_ob[i] = len(df[df.TH_OB == [i]])
    
d_th_ob_tf = {}
for i in th_ob_tf_labels:
    d_th_ob_tf[i] = len(df[df.TH_OB_TF == [i]])

In [ ]:
## swapping codes for number of occurrances
df.TH = [d_th[item] for item in df.TH]  
df['pTH'] = df.TH / outcomes

df.TH_OB = [d_th_ob[item] for item in df.TH_OB]
df['pTH_OB'] = df.TH_OB / df.TH

df.TH_OB_TF = [d_th_ob_tf[item] for item in df.TH_OB_TF]
df['pTH_OB_TF'] = df.TH_OB_TF / df.TH_OB

## Calculating result probability
df['pTotal'] = df.pTH * df.pTH_OB * df.pTH_OB_TF


In [ ]:
## Creating prob dataframe to drop duplicates
prob = df.drop(['Depth', 'HC_Column', 'Trap_height', 'Trap_Fill_2%'], axis=1)
prob.drop_duplicates(inplace=True)

## change type from category to object in order for sns.barplot correctly
for col in ['th_ob', 'th_ob_tf']:
    prob[col] = prob[col].astype('object')

In [ ]:
sns.jointplot(y='HC_Column', x='Trap_height', data=df, kind='reg',)

In [ ]:
## Plot results : using FacetGrid
g = sns.FacetGrid(data=prob, row='OB' , col='th' )
g.map(plt.bar, 'th_ob_tf', 'pTotal')
plt.savefig('overburden_HCcolumn_by_trapfill_categoryv2',bbox_inches='tight')

In [ ]:
## Plot results using subplots
fig, axes = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(12,12))
#plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

sns.barplot(x='th_ob_tf', y='pTotal', data=prob[prob['th_ob']=='A1'], ax=axes[0,0])
sns.barplot(x='th_ob_tf', y='pTotal', data=prob[prob['th_ob']=='A2'], ax=axes[1,0])
sns.barplot(x='th_ob_tf', y='pTotal', data=prob[prob['th_ob']=='A3'], ax=axes[2,0])
sns.barplot(x='th_ob_tf', y='pTotal', data=prob[prob['th_ob']=='B1'], ax=axes[0,1])
sns.barplot(x='th_ob_tf', y='pTotal', data=prob[prob['th_ob']=='B2'], ax=axes[1,1])
sns.barplot(x='th_ob_tf', y='pTotal', data=prob[prob['th_ob']=='B3'], ax=axes[2,1])
sns.barplot(x='th_ob_tf', y='pTotal', data=prob[prob['th_ob']=='C1'], ax=axes[0,2])
sns.barplot(x='th_ob_tf', y='pTotal', data=prob[prob['th_ob']=='C2'], ax=axes[1,2])
sns.barplot(x='th_ob_tf', y='pTotal', data=prob[prob['th_ob']=='C3'], ax=axes[2,2])

#axes[2,2].set_title('Trap Height (>300m)')
#axes[2,2].set_xlabel('Trap Fill (m)')
#axes[0,0].set_ylabel('Probability')
#axes[2,2].legend()

plt.savefig('overburden_HCcolumn_by_trapfill_category',bbox_inches='tight')